In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cufflinks as cf

data_dir = Path('../data')

income_groups_1639 = pd.read_csv(data_dir / 'interim' / 'income_groups_1639.csv')
income_1880 = pd.read_csv(data_dir / 'interim' / 'income_tax_records_1880.csv')

In [ ]:
print(income_1880)
print(income_groups_1639)